**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video

You are using pip version 19.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, PReLU, Flatten

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function act returns the next action to do. 
>- If the agent is already trained, it chooses the "best" action (i.e. the one that was learned during the training phase in order to maximize the reward).
>- If the agent is not trained yet, it will act with the "best" action with a probability $1-\epsilon$ and will take a random action with a probability $\epsilon$.
>
> Doing this during the training phase is essential in order to guarantee exploration: the agent must explore the actions in order to learn about them and improve its chances of getting future reward.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [9]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 10 # set small when debugging
epochs_test= 10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

>We can first notice that lines and columns are added at the border of the arrays ```position``` and ```board``` so that the grid seen by the mouse is always of size 5x5 (even if the mouse is at the limit of the grid). 
>
>In these margins, the value of ```position``` is set to -1, meaning that the position is invalid. Furthermore, the ```position``` array is set to 1 in x,y and 0 otherwise. 
>
>The array ```board``` contains the reward associated to each position (+0.5 for cheese, -1 for poison and 0 for empty or margin slot)

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))
    
    
    

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.0/12.0. Average score (-6.0)
Win/lose count 9.0/12.0. Average score (-4.5)
Win/lose count 10.0/11.0. Average score (-3.3333333333333335)
Win/lose count 9.0/18.0. Average score (-4.75)
Win/lose count 7.5/11.0. Average score (-4.5)
Win/lose count 5.5/8.0. Average score (-4.166666666666667)
Win/lose count 12.5/15.0. Average score (-3.9285714285714284)
Win/lose count 7.0/13.0. Average score (-4.1875)
Win/lose count 5.5/13.0. Average score (-4.555555555555555)
Win/lose count 13.5/13.0. Average score (-4.05)
Final score: -4.05


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




>We have that : $$Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> $$
>
>thus, $$ Q^\pi(s,a)=r(s, a) + E_{p^{\pi}}[\sum_{1\leq t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$$
>
>$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[\sum_{1\leq t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']|s_0=s, a_0=a]$$
>
>$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[\sum_{1\leq t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']]$$
>
>$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[\sum_{0\leq t\leq \infty}\gamma^{t+1}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']]$$
>
>$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[\gamma Q^{\pi}(s', a')]$$
>
>so finally : $$Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$ which shows the first question.
>
>Then, we have $$Q^{\pi}(s,a) = r(s,a) + \gamma \sum_{s^{'}}p(s^{'}|s,a) Q^{\pi}(s^{'},\pi(s^{'}))$$
>
>by taking the max over all policies $pi$, we get :
>
>\begin{align*}
    Q^{*}(s,a) &= r(s,a) + \gamma \max_{\pi} \sum_{s'}p(s^{'}|s,a) Q^{\pi}(s^{'},\pi(s^{'})) \\
               &= r(s,a) + \gamma \sum_{s^{'}}p(s^{'}|s,a) max_{\pi} Q^{\pi}(s^{'},\pi(s^{'})) \\
               &= r(s,a) + \gamma \sum_{s^{'}}p(s^{'}|s,a) Q^{*}(s^{'},\pi^{*}(s^{'})) \\
               &= r(s,a) + \gamma \sum_{s^{'}}p(s^{'}|s,a) max_{a^{'}}Q^{*}(s^{'},a^{'}) \\
               &= r(s,a) + \gamma E_{s^{'}\sim p(.|s,a)}[max_{a^{'}}Q^{*}(s^{'},a^{'})]
\end{align*}
>
>which shows the second point.
>
>We parametrize $Q$ by $\theta$, and we want to find that $\theta$ >that minimizes the following equation :
>
>$$\mathcal{L}(\theta)= (Q^{*}(s,a)-Q_{\theta}(s,a))^{2}$$
>
>But since $Q^{*}$ is unknownwe approximate it by its one-step estimate $y_{t} = r(s,a) +\gamma E_{s^{'}\sim p(.|s,a)}[\max_{a'}Q_{\theta_{t}}(s',a')]$.
>
>We obtain the following equation :
>
>$$\mathcal{L}(\theta_{t+1})=E_{\mathcal{D}}[(y_{t}-Q_{\theta_{t+1}}(s,a))^{2}]$$ from which we deduce the final point.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.choice(len(self.memory))]

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s.reshape((1,5,5,self.n_state))))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size,5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            
            target_q[i] = self.model.predict(s_.reshape((1,5,5,self.n_state)))
            Q_sa = np.max(self.model.predict(n_s_.reshape((1,5,5,self.n_state))))
            
            if game_over_:
                ######## FILL IN
                target_q[i, a_] = r_
            else:
                ######## FILL IN
                target_q[i, a_] = r_ + self.discount*Q_sa
                
                
        ######## FILL IN
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)
        
        l = self.model.train_on_batch(input_states, target_q)
        
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state)))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(4, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        #model.summary()
        self.model = model

In [15]:
epochs_train= 51 # set small when debugging
epochs_test= 10 # set small when debugging

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=0.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train50.mp4'))

Epoch 000/051 | Loss 0.0031 | Win/lose count 3.5/5.0 (-1.5)
Epoch 001/051 | Loss 0.0214 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/051 | Loss 0.0082 | Win/lose count 6.0/5.0 (1.0)
Epoch 003/051 | Loss 0.0018 | Win/lose count 1.5/5.0 (-3.5)
Epoch 004/051 | Loss 0.0025 | Win/lose count 4.5/3.0 (1.5)
Epoch 005/051 | Loss 0.0018 | Win/lose count 3.0/4.0 (-1.0)
Epoch 006/051 | Loss 0.0069 | Win/lose count 2.5/5.0 (-2.5)
Epoch 007/051 | Loss 0.0053 | Win/lose count 3.5/3.0 (0.5)
Epoch 008/051 | Loss 0.0024 | Win/lose count 8.0/5.0 (3.0)
Epoch 009/051 | Loss 0.0044 | Win/lose count 7.5/5.0 (2.5)
Epoch 010/051 | Loss 0.0059 | Win/lose count 8.0/3.0 (5.0)
Epoch 011/051 | Loss 0.0058 | Win/lose count 5.0/5.0 (0.0)
Epoch 012/051 | Loss 0.0562 | Win/lose count 8.5/2.0 (6.5)
Epoch 013/051 | Loss 0.0032 | Win/lose count 7.5/3.0 (4.5)
Epoch 014/051 | Loss 0.0410 | Win/lose count 9.5/7.0 (2.5)
Epoch 015/051 | Loss 0.0054 | Win/lose count 9.5/5.0 (4.5)
Epoch 016/051 | Loss 0.0499 | Win/lose count 9.0/4.

In [16]:
test(agent,env,epochs_test,prefix='fc_test')
HTML(display_videos('fc_test0.mp4'))

Win/lose count 8.5/3.0. Average score (5.5)
Win/lose count 3.5/0. Average score (4.5)
Win/lose count 9.5/3.0. Average score (5.166666666666667)
Win/lose count 7.5/2.0. Average score (5.25)
Win/lose count 14.5/5.0. Average score (6.1)
Win/lose count 7.0/1.0. Average score (6.083333333333333)
Win/lose count 5.0/0. Average score (5.928571428571429)
Win/lose count 6.0/0. Average score (5.9375)
Win/lose count 1.0/6.0. Average score (4.722222222222222)
Win/lose count 4.5/2.0. Average score (4.5)
Final score: 4.5


> I used an Adam optimizer because I noticed a convergence that was smoother and quicker than with sgd. The a good value for the learning rate for this optimizer is 0.001.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [17]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Sequential()
        model.add(Conv2D(8, kernel_size = 2, strides = (1,1), padding = 'valid', input_shape = (5,5,self.n_state), activation = 'relu'))
        model.add(Conv2D(16, kernel_size = 2, strides = (1,1), padding = 'valid', activation = 'relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        #model.summary()
        self.model = model

In [18]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train50.mp4'))

Epoch 000/051 | Loss 0.0032 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/051 | Loss 0.0066 | Win/lose count 3.0/7.0 (-4.0)
Epoch 002/051 | Loss 0.0046 | Win/lose count 4.5/7.0 (-2.5)
Epoch 003/051 | Loss 0.0029 | Win/lose count 8.5/4.0 (4.5)
Epoch 004/051 | Loss 0.0052 | Win/lose count 10.0/3.0 (7.0)
Epoch 005/051 | Loss 0.0024 | Win/lose count 17.0/4.0 (13.0)
Epoch 006/051 | Loss 0.0048 | Win/lose count 7.0/2.0 (5.0)
Epoch 007/051 | Loss 0.0521 | Win/lose count 15.0/6.0 (9.0)
Epoch 008/051 | Loss 0.0443 | Win/lose count 12.0/2.0 (10.0)
Epoch 009/051 | Loss 0.0133 | Win/lose count 6.0/5.0 (1.0)
Epoch 010/051 | Loss 0.0581 | Win/lose count 10.0/3.0 (7.0)
Epoch 011/051 | Loss 0.0016 | Win/lose count 4.0/2.0 (2.0)
Epoch 012/051 | Loss 0.0034 | Win/lose count 10.0/4.0 (6.0)
Epoch 013/051 | Loss 0.0019 | Win/lose count 5.5/0 (5.5)
Epoch 014/051 | Loss 0.0016 | Win/lose count 10.5/1.0 (9.5)
Epoch 015/051 | Loss 0.0020 | Win/lose count 1.5/4.0 (-2.5)
Epoch 016/051 | Loss 0.0019 | Win/lose count 

In [19]:
test(agent,env,epochs_test,prefix='cnn_test')
HTML(display_videos('cnn_test0.mp4'))

Win/lose count 14.5/1.0. Average score (13.5)
Win/lose count 14.5/6.0. Average score (11.0)
Win/lose count 15.0/2.0. Average score (11.666666666666666)
Win/lose count 15.0/5.0. Average score (11.25)
Win/lose count 15.5/2.0. Average score (11.7)
Win/lose count 11.0/4.0. Average score (10.916666666666666)
Win/lose count 17.5/5.0. Average score (11.142857142857142)
Win/lose count 18.0/3.0. Average score (11.625)
Win/lose count 20.0/4.0. Average score (12.11111111111111)
Win/lose count 7.5/1.0. Average score (11.55)
Final score: 11.55


***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=0.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=0.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 10.0/3.0. Average score (7.0)
Win/lose count 3.5/3.0. Average score (3.75)
Win/lose count 14.5/6.0. Average score (5.333333333333333)
Win/lose count 13.5/4.0. Average score (6.375)
Win/lose count 7.5/1.0. Average score (6.4)
Win/lose count 10.0/2.0. Average score (6.666666666666667)
Win/lose count 15.5/3.0. Average score (7.5)
Win/lose count 9.5/0. Average score (7.75)
Win/lose count 15.0/1.0. Average score (8.444444444444445)
Win/lose count 19.5/4.0. Average score (9.15)
Final score: 9.15
Test of the FC
Win/lose count 8.0/4.0. Average score (4.0)
Win/lose count 6.0/2.0. Average score (4.0)
Win/lose count 11.5/4.0. Average score (5.166666666666667)
Win/lose count 11.5/2.0. Average score (6.25)
Win/lose count 4.5/3.0. Average score (5.3)
Win/lose count 5.0/3.0. Average score (4.75)
Win/lose count 8.0/0. Average score (5.214285714285714)
Win/lose count 3.5/2.0. Average score (4.75)
Win/lose count 7.5/1.0. Average score (4.944444444444445)
Win/lose count 6.5

In [ ]:
HTML(display_videos('cnn_test9.mp4'))

In [ ]:
HTML(display_videos('fc_test3.mp4'))

>On average, we observe that the results are better with the CNN than the FC agent. However, the agent still need to be improved because it is not exploring the map enough, and tend to oscillate too much.
>
>In the next cells, we compute the same experiments, but we increase the temperature in the test phase.

In [33]:
env = Environment(grid_size=size, max_time=T,temperature=0.7)
agent_cnn = DQN_CNN(size, lr=0.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=0.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 30.0/2.0. Average score (28.0)
Win/lose count 45.5/6.0. Average score (33.75)
Win/lose count 32.5/2.0. Average score (32.666666666666664)
Win/lose count 27.5/6.0. Average score (29.875)
Win/lose count 33.0/5.0. Average score (29.5)
Win/lose count 14.5/3.0. Average score (26.5)
Win/lose count 28.0/4.0. Average score (26.142857142857142)
Win/lose count 37.5/5.0. Average score (26.9375)
Win/lose count 44.5/2.0. Average score (28.666666666666668)
Win/lose count 31.5/3.0. Average score (28.65)
Final score: 28.65
Test of the FC
Win/lose count 21.5/3.0. Average score (18.5)
Win/lose count 21.0/5.0. Average score (17.25)
Win/lose count 18.0/3.0. Average score (16.5)
Win/lose count 38.0/2.0. Average score (21.375)
Win/lose count 19.0/5.0. Average score (19.9)
Win/lose count 19.5/1.0. Average score (19.666666666666668)
Win/lose count 34.0/6.0. Average score (20.857142857142858)
Win/lose count 23.0/4.0. Average score (20.625)
Win/lose count 17.5/6.0. Average score (

In [36]:
HTML(display_videos('cnn_test1.mp4'))

In [37]:
HTML(display_videos('fc_test3.mp4'))

>Several remarks:
>
> - The CNN agent performs again better on average than the FC agent.
>
> - We notice that the agent suffers less from exploration problems with a higher temperature
>
> - However, this second point is most probably caused by the fact that augmenting the temperature also increases the ratio cheese/poison, and thus, gives less chance for the agent to be stuck in an oscillation. This remark therefore motivates the following section.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [38]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    final_eps = 0.01
    decr_eps = (agent.epsilon-final_eps)/epoch
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        #Decrease epsilon
        agent.set_epsilon(agent.epsilon-decr_eps)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
        
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        reward = 0
        if train:
            reward += -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        self.t = self.t + 1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size,1)), axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state

In [39]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.001, epsilon = 0.2, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/051 | Loss 0.0096 | Win/lose count 3.0/10.199999999999987 (-7.199999999999987)
Epoch 001/051 | Loss 0.0126 | Win/lose count 4.5/10.099999999999984 (-5.599999999999984)
Epoch 002/051 | Loss 0.0221 | Win/lose count 3.5/12.299999999999983 (-8.799999999999983)
Epoch 003/051 | Loss 0.0078 | Win/lose count 2.5/10.399999999999979 (-7.899999999999979)
Epoch 004/051 | Loss 0.0098 | Win/lose count 5.0/9.199999999999987 (-4.199999999999987)
Epoch 005/051 | Loss 0.0054 | Win/lose count 6.5/8.299999999999992 (-1.7999999999999918)
Epoch 006/051 | Loss 0.0043 | Win/lose count 9.0/7.1999999999999975 (1.8000000000000025)
Epoch 007/051 | Loss 0.0056 | Win/lose count 14.5/8.699999999999996 (5.800000000000004)
Epoch 008/051 | Loss 0.0150 | Win/lose count 11.5/9.799999999999988 (1.7000000000000117)
Epoch 009/051 | Loss 0.0145 | Win/lose count 16.5/8.499999999999998 (8.000000000000002)
Epoch 010/051 | Loss 0.0594 | Win/lose count 6.5/7.799999999999988 (-1.2999999999999883)
Epoch 011/051 | Loss 0.0

In [40]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 10.0/1.0. Average score (9.0)
Win/lose count 9.0/0. Average score (9.0)
Win/lose count 21.0/0. Average score (13.0)
Win/lose count 13.0/0. Average score (13.0)
Win/lose count 16.5/1.0. Average score (13.5)
Win/lose count 8.5/0. Average score (12.666666666666666)
Win/lose count 16.5/2.0. Average score (12.928571428571429)
Win/lose count 18.5/0. Average score (13.625)
Win/lose count 11.5/1.0. Average score (13.277777777777779)
Win/lose count 19.5/0. Average score (13.9)
Final score: 13.9


>These improvements drastically leverage the lack of exploration problem, as we can see in this final video.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***